# Fraud Detection Workflow with Transformers

This notebook implements an agentic workflow for fraud detection using LangGraph and local Transformers models.

In [ ]:
# Install required packages
!pip install -q langgraph langchain-core langchain-community transformers accelerate bitsandbytes google-colab

# Import libraries

pip install litellm

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_community.chat_models import ChatLiteLLM
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from typing import Annotated, Sequence
import operator
llm = ChatLiteLLM(model="ollama/mistral",temperature=0.7)
tools = [calculator]
llm_with_tools = llm.bind_tools(tools)
# Define tools
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b


# Augment the LLM with tools
tools = [add, multiply, divide]
tools_by_name = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)

In [19]:
from langgraph.graph import MessagesState
from langchain_core.messages import ToolMessage, SystemMessage # Import SystemMessage
from typing import Literal # Import Literal

# Nodes
def llm_call(state: MessagesState):
    """LLM decides whether to call a tool or not"""

    return {
        "messages": [
            llm_with_tools.invoke(
                [
                    SystemMessage(
                        content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
                    )
                ]
                + state["messages"]
            )
        ]
    }


def tool_node(state: dict):
    """Performs the tool call"""

    result = []
    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    return {"messages": result}


# Conditional edge function to route to the tool node or end based upon whether the LLM made a tool call
def should_continue(state: MessagesState) -> Literal["environment", END]:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""

    messages = state["messages"]
    last_message = messages[-1]
    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "Action" # Should be "tools" as per graph node name
    # Otherwise, we stop (reply to the user)
    return END

In [ ]:

# Build graph
workflow = StateGraph(MessagesState)

# Add nodes
workflow.add_node("llm_call", llm_call)
workflow.add_node("tool_node", tool_node)

# Set entry point
workflow.set_entry_point("llm_call")

# Add edges
workflow.add_conditional_edges(
    "llm_call",
    should_continue,
    {
        "Action": "tool_node", # Route to tool_node if the LLM calls a tool
        END: END # End if the LLM does not call a tool
    }
)
workflow.add_edge("tool_node", "llm_call") # After executing tools, call the LLM again

# Compile the graph
app = workflow.compile()